In [ ]:
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\User\AppData\Local\Tesseract-OCR\tesseract.exe'
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

token = "secret_9nQZ4zboj7lWTzMDg"
databaseId = "e609d6f8b1c"

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

def readDatabase(databaseId, headers):
  readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"

  res = requests.request("POST", readUrl, headers=headers)
  data = res.json()
  # print(res.status_code)
  # print(res.text)

  with open('./Prayer_Calendar_2.json', 'w', encoding='utf8') as f:
    json.dump(data, f, ensure_ascii=False)

def createPage(databaseId, headers, properties):
    createUrl = 'https://api.notion.com/v1/pages'

    newPageData = {
      "object": "page",
      "id": "40b185b1-feb6-4c17-a615-958abe77fb6d",
      "created_time": "2022-09-25T13:49:00.000Z",
      "last_edited_time": "2022-09-25T13:51:00.000Z",
      "created_by": {
        "object": "user",
        "id": "a0d7d6f1-4611-4b95-afb3-6cc503a70ad6"
      },
      "last_edited_by": {
        "object": "user",
        "id": "a0d7d6f1-4611-4b95-afb3-6cc503a70ad6"
      },
      "cover": None,
      "icon": None,
      "parent": {
        "type": "database_id",
        "database_id": databaseId
      },
      "archived": False,
      "properties": properties,
      "url": "https://www.notion.so/Asr-Jammah-40b185b1feb64c17a615958abe77fb6d"
    }
      
    data = json.dumps(newPageData)
    # print(str(uploadData))

    res = requests.request("POST", createUrl, headers=headers, data=data)

    print(res.status_code)
    print(res.text)

# img = cv2.imread(r'C:\Users\User\Downloads\January_2023.png', 0)
img = cv2.imread('January_2023.png', 0)

# crop = img[900:img.shape[0]-500, 0:img.shape[1]] #1070
crop = img[1260:img.shape[0]-400, 0:img.shape[1]]

gray = cv2.threshold(crop,200,255,cv2.THRESH_BINARY)[1]

cal = list(filter(None, pytesseract.image_to_string(gray, config=r'--oem 3 --psm 3 -c tessedit_char_blacklist==.‘,()').split("\n")))

plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

# for i in cal:
#   print(i)

# title = cal[0]; headings = cal[1]
title = "January 2023"

Day = {
    "MO": "Mon",
    "TU": "Tue",
    "WE": "Wed",
    "TH": "Thu",
    "FR": "Fri",
    "SA": "Sat",
    "SU": "Sun",
    "su": "Sun"
}

readDatabase(databaseId, headers)

Fajr = []; FajrJ = []; Shurooq = []; Dhuhr = []; DhuhrJ = []; Asr = []; AsrJ = []; Maghrib = []; MaghribJ = []; Isha = []; IshaJ = []; days = []

for i in cal:
    day = str(Day[i.split()[0][:2]])

    if len(i.split()[0]) == 2:
        month = i.split()[1]
    else:
        if i.split()[0][-2] == "O":
            month = "0" + i.split()[0][-1]
        else:
            month = i.split()[0][-2:]
    
    def date(prayer):
        # print(title)
        # print(day)
        # print(month)
        # print(prayer)
        return str(datetime.strptime(title + " " + day + " " + month + " " + prayer, '%B %Y %a %d %H:%M'))
        #"January 2023, Tue, 03, 06:12" → "2023-01-03 06:12:00"

    days.append(day + " " +  month + " " +  title)
    Fajr.append(date(i.split()[-11]))
    FajrJ.append(date(i.split()[-10]))
    Shurooq.append(date(i.split()[-9]))
    Dhuhr.append(date(i.split()[-8]))
    DhuhrJ.append(date(i.split()[-7]))
    Asr.append(date(i.split()[-6]))
    AsrJ.append(date(i.split()[-5]))
    Maghrib.append(date(i.split()[-4]))
    MaghribJ.append(date(i.split()[-3]))
    Isha.append(date(i.split()[-2]))
    IshaJ.append(date(i.split()[-1]))

Timings = {
  "Fajr": Fajr,
  "Fajr Jammah": FajrJ,
  "Shurooq": Shurooq,
  "Dhuhr": Dhuhr,
  "Dhuhr Jammah": DhuhrJ,
  "Asr": Asr,
  "Asr Jammah": AsrJ,
  "Maghrib": Maghrib,
  "Maghrib Jammah": MaghribJ,
  "Isha": Isha,
  "Isha Jammah": IshaJ
}

# print(Timings)

properties = {}

for p in Timings:
  
  for i in range(len(days)):
    properties[f"{i+1:02d}/01/23"] = {
      "id": "Y~bC",
      "type": "date",
      "date": {
        "start": Timings[p][i],
        "end": None,
        "time_zone": "Europe/London"
      }
    }

  for i in range(31-len(days)):
    properties[f"{31-i:02d}/01/23"] = {
      "id": "Y~bC",
      "type": "date",
      "date": None
    }

  properties["Name"] = {
    "id": "title",
    "type": "title",
    "title": [{
      "type": "text",
      "text": { "content": p, "link": None },
      "annotations": {
        "bold": False,
        "italic": False,
        "strikethrough": False,
        "underline": False,
        "code": False,
        "color": "default"
      },
      "plain_text": p,
      "href": None
    }]
  }

  # print(properties)
  # print("")

  createPage(
    databaseId,
    headers,
    properties
  )


In [ ]:
import requests
import json

token = "secret_9WTzMDg"
databaseId = "e60948b1c" #code

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

def updatePage(databaseId, headers):
    data = json.load(open("Prayer_Calendar_2.json"))

    updateUrl = f"https://api.notion.com/v1/databases/{databaseId}"

    updateData = {
        "properties": properties
    }

    data = json.dumps(updateData)

    response = requests.request("PATCH", updateUrl, headers=headers, data=data)

    print(response.status_code)
    print(response.text)

properties = {}

for i in range(31):
    properties[f"{i+1:02d}/01/23 "] = {
        "name": f"{i+1:02d}/02/23 "
    }

    properties[f"{i+1:02d}/01/23"] = {
        "name": f"{i+1:02d}/02/23"
    }

updatePage(databaseId, headers)